### This notebook consist of code for creating the html files for the website each time data is updated.

##### Set-up

In [33]:
#Importing classes from the Jinja2 library to load and render templates.
import json
import os
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError
from pathlib import Path


In [34]:
os.getcwd()

'/Users/harshul/website clone/harshul/test/groupwebsite_generator/notebooks'

In [35]:
current_dir = Path(os.getcwd())

MEMBERS_DIR_PATH = current_dir / '../../group-data/members/'
WEBSITE_DATA_PATH = current_dir / '../../group-data/website_data/'
CONTENT_DIR_PATH = current_dir / '../../group-data/website_data/content/'


In [36]:
#Function for creating proper html file names
def page_link(a):
    if ' ' in a:
        return a.replace(' ', '_')
    else:
        return a

In [37]:
member_records = []

for member_dir in MEMBERS_DIR_PATH.glob('*'):
    
    if member_dir.name.startswith('.'):
        continue
    
    member_record = json.load(open(member_dir / 'info.json'))
    member_json_dir = member_dir / 'jsons'
    
    top_experience_role = ""
    top_education_degree = ""
    top_education_end_date = None
  
    if (member_experiences := (member_json_dir / 'experiences.json')).exists():
        experiences = json.load(open(member_experiences))
        if experiences:
            top_experience_role = experiences[0].get('role', "")
    
    if (member_education := (member_json_dir / 'education.json')).exists():
        education = json.load(open(member_education))
        if education:
            top_education_degree = education[0].get('degree', "")
            top_education_end_date = education[0].get('end_date')
    
    if (member_projects := (member_json_dir / 'projects.json')).exists():
        current_position = json.load(open(member_projects))[0]
        member_record.update(current_position)
    
    if (member_links := (member_json_dir / 'social_links.json')).exists():
        social_links = json.load(open(member_links))
        member_record.update(social_links)
    
    # For storing academic role
    if 'Assistant Professor' in top_experience_role or 'Professor' in top_experience_role:
        academic_role = 'Professor'
    elif top_experience_role in ['Visiting Researcher', 'Postdoctoral Researcher']:
        academic_role = 'Postdoctoral researcher'
    elif top_education_degree == 'Bachelors':
        academic_role = 'Graduate Student' if top_education_end_date is not None else 'Undergraduate Student'
    elif top_education_degree in ['Masters', 'PhD']:
        academic_role = 'Graduate Student'
    else:
        academic_role = 'Other'

    
    member_record['academic_role'] = academic_role
    member_records.append(member_record)

member_df = pd.DataFrame(member_records)
people_page_df = member_df[['first_name', 'last_name', 'academic_role', 'image_path', 'project_title', 'website', 'github_handle', 'twitter_handle', 'linkedin_handle', 'email', 'orcid']]


In [38]:
# To view dict structure.
people_page_df.to_dict(orient='records')

[{'first_name': 'Joshua',
  'last_name': 'Shields',
  'academic_role': 'Graduate Student',
  'image_path': 'media/images/josh_photo.jpg',
  'project_title': 'Surviving Companions Of Supernovae And Stellar Atmospheric Modeling',
  'website': 'https://jvshields.github.io/',
  'github_handle': 'jvshields',
  'twitter_handle': '',
  'linkedin_handle': '',
  'email': 'shield90@msu.edu',
  'orcid': '0000-0002-1560-5286'},
 {'first_name': 'Anirban ',
  'last_name': 'Dutta',
  'academic_role': 'Postdoctoral researcher',
  'image_path': 'media/images/anirban_dutta.jpg',
  'project_title': 'Non-LTE modeling of supernova spectra',
  'website': 'https://sites.google.com/view/anirbaniamdutta',
  'github_handle': 'Knights-Templars',
  'twitter_handle': 'Anirban29Dutta',
  'linkedin_handle': 'anirban-dutta-6a0377238',
  'email': 'anirbaniamdutta@gmail.com',
  'orcid': '0000-0002-7708-3831'},
 {'first_name': 'Erin',
  'last_name': 'Visser',
  'academic_role': 'Undergraduate Student',
  'image_path': '

In [39]:
# To view dataframe
people_page_df


,first_name,last_name,academic_role,image_path,project_title,website,github_handle,twitter_handle,linkedin_handle,email,orcid
0,Joshua,Shields,Graduate Student,media/images/josh_photo.jpg,Surviving Companions Of Supernovae And Stellar...,https://jvshields.github.io/,jvshields,,,shield90@msu.edu,0000-0002-1560-5286
1,Anirban,Dutta,Postdoctoral researcher,media/images/anirban_dutta.jpg,Non-LTE modeling of supernova spectra,https://sites.google.com/view/anirbaniamdutta,Knights-Templars,Anirban29Dutta,anirban-dutta-6a0377238,anirbaniamdutta@gmail.com,0000-0002-7708-3831
2,Erin,Visser,Undergraduate Student,media/images/erin_visser_website_pic.jpg,NaN,,erinvisser,,,visserer@msu.edu,0009-0001-8470-275X
3,Vicente,Amado Olivo,Graduate Student,media/images/ESD_headshot.jpg,Development Of A Global Registry For Peer Revi...,,,vamadolivo,,amadovic@msu.edu,0000-0003-2248-0941
4,Yuki,Matsumura,Graduate Student,media/images/yuki_face.png,Type IIP Supernovae As Cosmological Distance P...,,ymatsumu,,,matsumurayuki725@gmail.com,
5,Andrew,Fullard,Graduate Student,media/images/me.jpg,Inferring explosion conditions from late-time ...,NaN,andrewfullard,astrofullard,andrew-fullard-a9a487168/,fullarda@msu.edu,0000-0001-7343-1678
6,Isaac,Smith,Graduate Student,media/images/isaac_image.jpg,NaN,,smithis7,,,smithis7@msu.edu,0000-0003-0440-3918
7,Hayden,Monk,Undergraduate Student,media/images/hayden.jpg,Surviving Companion Search In SNR-0509,NaN,NaN,NaN,NaN,NaN,NaN
8,Atharva,Arya,Graduate Student,media/images/atharva.jpg,Mitigating open science sustainability issues,https://www.atharvaarya.tech/,atharva-2001,2001_atharva,atharva-arya,aryaatharva18@gmail.com,
9,Richard,Dow,Undergraduate Student,media/images/richard.jpg,Identifying Worldwide Astrophysicists From Sci...,,prettytrippy,,,dow00019@umn.edu,0009-0007-2842-1690


In [40]:



website_data_files = {
    'contact': 'contact.json',
    'general': 'general.json',
    'research': 'research.json',
    'homepage': 'homepage.json',
    'support': 'support.json'
}


contact, general, research, homepage, support = [{}] * 5


for var_name, webpage_json in website_data_files.items():
    file_path = WEBSITE_DATA_PATH / webpage_json
    if file_path.exists():
        with open(file_path, 'r') as f:
            locals()[var_name] = json.load(f)


In [41]:
contact_df = pd.DataFrame.from_records([contact])
general_df = pd.DataFrame.from_records([general])
homepage_df = pd.DataFrame.from_records([homepage])
research_df = pd.DataFrame.from_records([research])
support_df = pd.DataFrame.from_records([support])


In [42]:

content_dfs = []  
for json_file in CONTENT_DIR_PATH.glob('*.json'):
    with open(json_file, 'r') as file:
        data = json.load(file)
        
       
        df = pd.json_normalize(data)
        
        content_dfs.append(df)


content_df = pd.concat(content_dfs, ignore_index=True, sort=False)




In [43]:
#Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates
environment = Environment(loader=FileSystemLoader('templates/'),extensions=['jinja2.ext.loopcontrols'])
environment.globals['page_link'] = page_link
environment.globals['ContentData'] = content_df

In [44]:
research_content_unsorted = content_df[content_df['tags'].apply(lambda x: any('research' in tag for tag in x))]
research_content = research_content_unsorted.groupby('category').apply(lambda x: x.sort_values('article_date', ascending=False)).reset_index(drop=True)

In [45]:
news_content_unsorted = content_df[content_df['tags'].apply(lambda x: any('news' in tag for tag in x))]
news_content = news_content_unsorted.sort_values(by="article_date", ascending=False)

In [46]:
latest_content_df = pd.DataFrame()

for category in content_df.category.unique():
    latest_data = pd.Series(content_df[content_df.category == category].iloc[0])
    latest_content_df = latest_content_df._append(latest_data, ignore_index=True)

latest_content_df['article_date'] = pd.to_datetime(latest_content_df['article_date'], format='%m-%d-%Y')
latest_content_df = latest_content_df.sort_values(by='article_date', ascending=False)

##### Homepage

In [47]:
homepage_template = environment.get_template('homepage.html.j2')

TemplateNotFound: homepage.html.j2

In [ ]:
general_dict = general_df.iloc[0].to_dict()
homepage_dict = homepage_df.iloc[0].to_dict()


In [ ]:
homepage_content = homepage_template.render(general=general_dict,
                                            homepage=homepage_dict,
                                            recent_content=latest_content_df.to_dict(orient='records'))


In [ ]:
with open('../kerzendorf-group.github.io/index.html', mode='w', encoding='utf-8') as Homepage:
    Homepage.write(homepage_content)

##### People Page

In [ ]:
templates_dir_path = current_dir / '../templates/'
environment = Environment(loader=FileSystemLoader(str(templates_dir_path)))
people_template = environment.get_template("people.html.j2")


In [ ]:
people_template = environment.get_template("people.html.j2")

In [ ]:
environment.globals['page_link'] = page_link
people_content = people_template.render(general=general_dict, present_members=people_page_df, result=result)


In [ ]:
with open("../../kerzendorf-group.github.io/People.html", mode="w", encoding="utf-8") as people:
    people.write(people_content)

##### Individual People Page

In [ ]:
ind_person_template = environment.get_template("individual_person.html.j2")

In [ ]:
for person in people_df['id']:
            filename = f"../kerzendorf-group.github.io/members/{ person }/{ person }.html"
            ind_person_content = ind_person_template.render(general=data["general"], 
                                                            member_id=person, 
                                                            content=content_df.to_dict(orient='records'))
            with open(filename, mode="w", encoding="utf-8") as page:
                page.write(ind_person_content)

##### Research Page

In [ ]:
research_template = environment.get_template("research.html.j2")

In [ ]:
main_page_research_content = research_template.render(general=data["general"],
                                            content=research_content)

In [ ]:
with open("../kerzendorf-group.github.io/Research.html", mode="w", encoding="utf-8") as research:
        research.write(main_page_research_content)

In [ ]:
sub_research_template = environment.get_template("sub_research_frontpage.html.j2")

In [ ]:
for category in content_df.loc[content_df.category != "News", "category"].unique():
        sub_research_content = sub_research_template.render(general=data["general"], 
                                                            research_general=data["research"], 
                                                            content = research_content,
                                                            category = category
                                                            )
        folder_path = f"../kerzendorf-group.github.io/sub_research/{page_link(category.lower())}"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"../kerzendorf-group.github.io/sub_research/{page_link(category.lower())}.html", mode="w", encoding="utf-8") as sub_research:
            sub_research.write(sub_research_content)

##### Individual Research Page

In [ ]:
template_no_twitter = environment.get_template("research_page_no_twitter.html.j2")

In [ ]:
for ind_research_keys, ind_research_values in research_content.iterrows():
    if "news" not in ind_research_values.category.lower():
        ind_research_content = template_no_twitter.render(general=data["general"], 
                                                          member_ids = people_df['id'],
                                                          nonmem_ids = non_mem_df['id'],
                                                          content = ind_research_values
                                                          
                                                            )
        folder_path = f"../kerzendorf-group.github.io/sub_research/{page_link(ind_research_values.category.lower())}"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"{ folder_path }/{page_link(ind_research_values.article_id.lower())}.html", mode="w", encoding="utf-8") as ind_research_page:
            ind_research_page.write(ind_research_content)

##### News Page

In [ ]:
news_content

In [ ]:
news_template = environment.get_template("news.html.j2")

In [ ]:
news_page_content = news_template.render(general=data["general"],
                                         content=news_content,
                                         member_ids=people_df['id'],
                                         nonmem_ids = non_mem_df['id'],
                                         category="News")

In [ ]:
with open("../kerzendorf-group.github.io/News.html", mode="w", encoding="utf-8") as news:
        news.write(news_page_content)

##### Individual News Pages

In [ ]:
news_template_no_twitter = environment.get_template("news_page_no_twitter.html.j2")
#news_template_twitter = environment.get_template("news_page_twitter.html.j2")

In [ ]:
for ind_news_keys, ind_news_values in news_content.iterrows():
        ind_news_content = news_template_no_twitter.render(general=data["general"], 
                                                          member_ids = people_df['id'],
                                                          nonmem_ids = non_mem_df['id'],
                                                          content = ind_news_values
                                                            )
        folder_path = f"../kerzendorf-group.github.io/news/"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"{ folder_path }/{page_link(ind_news_values.article_id.lower())}.html", mode="w", encoding="utf-8") as ind_news_page:
            ind_news_page.write(ind_news_content)

##### Support Page

In [ ]:
support_template = environment.get_template('support.html.j2')

In [ ]:
support_content = support_template.render(general=data["general"], support=data["support"])

In [ ]:
with open('../kerzendorf-group.github.io/Support.html', mode='w', encoding='utf-8') as support:
    support.write(support_content)

##### Contact

In [ ]:
contact_template = environment.get_template('contact.html.j2')

In [ ]:
contact_content = contact_template.render(general=data["general"], contact=data["contact"])

In [ ]:
with open('../kerzendorf-group.github.io/Contact.html', mode='w', encoding='utf-8') as contact:
    contact.write(contact_content)